In [7]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class RedditbotSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "redditbot"
    allowed_domains = ['www.reddit.com/r/gameofthrones/']
    start_urls = ['www.reddit.com/r/gameofthrones/']

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        #Extracting the content using css selectors
        titiles = response.css('.title.may-blank::text').extract()
        votes = rexponse.css('.score.unvoted::text').extract()
        
        for item in zip(titles, votes):
            #create a dictionary to store the scraped info
            scraped_info = {
                'title' : item[0],
                'vote' : item[1]
            }
            
            yield scraped_info
    
# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'csv',         # Store data in JSON format.
    'FEED_URI': 'firstscrapreddit.csv',  # Name our storage file.
    #'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(RedditbotSpider)
process.start()
print('Success!')


2018-10-30 12:44:23 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-10-30 12:44:23 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.5, cssselect 1.0.3, parsel 1.5.0, w3lib 1.19.0, Twisted 18.7.0, Python 3.6.3 |Anaconda, Inc.| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.0.3, Platform Windows-10-10.0.17134-SP0
2018-10-30 12:44:23 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'firstscrapreddit.csv'}
2018-10-30 12:44:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-10-30 12:44:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 's

ReactorNotRestartable: 

In [4]:
import pandas as pd

firstpage = pd.read_json('firstscrapreddit.csv', orient='records')
print(firstpage.shape)
print(firstpage.head())

ValueError: Expected object or value

Success!  Ten rows (representing ten entries) each with date, name, tags, and text.  


## Recursion

Now that we have a scraper that can pull the information we want off of a page and store it in a file, we want to run that scraper over _all_ the pages of the website. We do this using recursion – the Scrapy spider will run over a page, gather information, and then detect a link to the next page and _call itself_ on the new page.  Then the new instance of the spider will do the same thing. This will repeat until it arrives at a page with no link to a next page, or when it reaches a stopping condition we define.

Since _Everyday Sexism_ has a paginated structure (every page links to the page before it), we can use recursion to get all the entries on the site. If you wanted to scrape a website with a non-linear structure, you'd still use the recursive setup, but with different restrictions on what links the scraper should follow. Scrapy is efficient and will only scrape each page once, even if it travels to them multiple times.

You may notice we've added some additional arguments to the ´CrawlerProcess()´ command- these have to do with scraping etiquette and will be introduced in the next assignment.

Again, you'll want to restart the kernel.

In [ ]:
# Importing in each cell because of the kernel restarts.
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        'http://www.everydaysexism.com',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        # Get the URL of the previous page.
        next_page = response.xpath('//div[@class="nav-previous"]/a/@href').extract_first()
        
        # There are a LOT of pages here.  For our example, we'll just scrape the first 9.
        # This finds the page number. The next segment of code prevents us from going beyond page 9.
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        # Recursively call the spider to run on the next page, if it exists.
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)

# Tell the script how to run the crawler by passing in settings.
# The new settings have to do with scraping etiquette.          
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'data.json',       # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

In [ ]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf=pd.read_json('data.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())

# Conclusion

Nine pages at 10 entries a row gives us 90 rows - looks like we were successful in our scraping!  This is just a start. Scrapy is very flexible and will let you run multiple spiders with all sorts of different abilities.  In the next assignment, we'll talk about some of the options you can use to fine-tune scrapy and how to use your newfound scraping power in a responsible way.